In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# change for your username and password and CRUD Python module name
username = "aacuser"
password = "Shortrace2"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project Two')
app.add_external_link = True

#Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # customer's image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Place the HTML image tag in the line below into the app.layout code according to your design
#include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(id='customer-image',height='102',width='102',
                        src='data:image/png:base64,{}'.format(encoded_image.decode()),alt='customer image')),
    html.Hr(),
    html.Div(
        
#interactive filtering options with Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
    id= 'filter-type',
    options= [
        {'label':'Water Rescue','value': 1},
        {'label':'Mountain or Wilderness Rescue','value': 2},
        {'label':'Disaster Rescue','value': 3},
        {'label':'Reset', 'value': 4}
        
       ],
       value='MTL',
       labelStyle={'display':'inline-block'}
     
      )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #interactive data table 
        editable =False,
        filter_action="native",
        sort_action ="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current = 0,
        page_size = 10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### Add code to filter interactive data table with MongoDB queries
        if filter_type == '1':
        
           df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix",
           "Chesapeake Bay Retriever","Newfoundland"]},"sex_upon_outcome":"Intact Female",
           "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))

        elif filter_type == '2':
        
             df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepard",
             "Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))

        elif filter_type == '3':

             df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher",
             "German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})))


        elif filter_type == '4':

             df = pd.DataFrame.from_records(shelter.read({}))

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    dff = pd.Dataframe.from_dict(viewdata)
    
    new_df = dff.breed.value_counts()
    new_df.name = "breed"
    new_df = new_df.to_frame()
    new_df.reset_index(inplace = True)
    
    return [
        dcc.Graph(
        figure = px.pie(new_df, values = "breed", names = "index"))
    ]

@app.callback(
    Output('datatable-interactivity-map', "children"),
    [Input('datatable-interactivity', "derived_viewport_data"),
     Input('datatable-interactivity', "derived_viewport_selected_rows")])
def update_map(viewData,rows):
#code for geolocation chart
     viewDf = pd.DataFrame.from_dict(viewData)
     dff = viewDf.loc[row]
    # Austin TX is at [30.75.-976.48]
return[
        dl.Map(style = {'width': '1000px', 'height': '500px'}, center = [30.75,-97.48],zoom = 10,children =[
            dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and popup
            dl.Marker(position = [dff.loc[0,'location_lat'],dff.loc[0,'location_long']],children =[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        
    ])]


app

TypeError: object() takes no parameters